In [20]:
import requests
import http.client

import os
from dotenv import load_dotenv

import datetime

from bs4 import BeautifulSoup
import re
from fuzzywuzzy import fuzz

import spotipy
from spotipy.oauth2 import SpotifyOAuth

In [21]:
# get and format today's date
today = datetime.datetime.today().date().strftime('%Y-%m-%d')
today

'2022-02-07'

In [22]:
 # enter website info
url = f'https://www.bandsintown.com/?date={today}&date_filter=This+Week'
headers = {'User-Agent': 'USERAGENT'}

response = requests.get(url, headers=headers)

In [23]:
# check to see if we have soup...  ('200' means that we have a valid connection)
response.status_code

200

In [60]:
# start the soup
soup = BeautifulSoup(response.text, 'html.parser')

In [85]:
link_list = []

for link in soup.find_all('a', href=True, attrs={'href': re.compile(r"^https://www.bandsintown.com")}):
    # display the actual urls
    link_list.append(link.get('href'))

In [86]:
event_urls = [link for link in link_list if link.startswith("https://www.bandsintown.com/e/")]

In [87]:
event_urls

['https://www.bandsintown.com/e/102869786-martin-sexton-at-blue-note-hawaii?came_from=257&utm_medium=web&utm_source=home&utm_campaign=top_event',
 'https://www.bandsintown.com/e/102869786-martin-sexton-at-blue-note-hawaii?came_from=257&utm_medium=web&utm_source=home&utm_campaign=top_event',
 'https://www.bandsintown.com/e/102869786-martin-sexton-at-blue-note-hawaii?came_from=257&utm_medium=web&utm_source=home&utm_campaign=top_event',
 'https://www.bandsintown.com/e/102869786-martin-sexton-at-blue-note-hawaii?came_from=257&utm_medium=web&utm_source=home&utm_campaign=top_event',
 'https://www.bandsintown.com/e/1023510713-rebelution-at-waikiki-shell?came_from=257&utm_medium=web&utm_source=home&utm_campaign=top_event',
 'https://www.bandsintown.com/e/1023510713-rebelution-at-waikiki-shell?came_from=257&utm_medium=web&utm_source=home&utm_campaign=top_event',
 'https://www.bandsintown.com/e/1023510713-rebelution-at-waikiki-shell?came_from=257&utm_medium=web&utm_source=home&utm_campaign=top_e

In [88]:
# grab the 'event's near' info from bottom of website
pattern = re.compile(r"\b\S*-at\b")
reg_list = [re.search(pattern , str(x)).group(0) for x in event_urls]
print(reg_list)


['https://www.bandsintown.com/e/102869786-martin-sexton-at', 'https://www.bandsintown.com/e/102869786-martin-sexton-at', 'https://www.bandsintown.com/e/102869786-martin-sexton-at', 'https://www.bandsintown.com/e/102869786-martin-sexton-at', 'https://www.bandsintown.com/e/1023510713-rebelution-at', 'https://www.bandsintown.com/e/1023510713-rebelution-at', 'https://www.bandsintown.com/e/1023510713-rebelution-at', 'https://www.bandsintown.com/e/1023510713-rebelution-at', 'https://www.bandsintown.com/e/102724676-kt-tunstall-at', 'https://www.bandsintown.com/e/102724676-kt-tunstall-at', 'https://www.bandsintown.com/e/102724676-kt-tunstall-at', 'https://www.bandsintown.com/e/102724676-kt-tunstall-at', 'https://www.bandsintown.com/e/103102900-peter-white-at', 'https://www.bandsintown.com/e/103102900-peter-white-at', 'https://www.bandsintown.com/e/103102900-peter-white-at', 'https://www.bandsintown.com/e/103102900-peter-white-at', 'https://www.bandsintown.com/e/102977409-borgeous-at', 'https:/

In [89]:
# select the link from the original grab
reg_list = [x.replace('-at','').replace('-', ' ') for x in reg_list]
print(reg_list)

['https://www.bandsintown.com/e/102869786 martin sexton', 'https://www.bandsintown.com/e/102869786 martin sexton', 'https://www.bandsintown.com/e/102869786 martin sexton', 'https://www.bandsintown.com/e/102869786 martin sexton', 'https://www.bandsintown.com/e/1023510713 rebelution', 'https://www.bandsintown.com/e/1023510713 rebelution', 'https://www.bandsintown.com/e/1023510713 rebelution', 'https://www.bandsintown.com/e/1023510713 rebelution', 'https://www.bandsintown.com/e/102724676 kt tunstall', 'https://www.bandsintown.com/e/102724676 kt tunstall', 'https://www.bandsintown.com/e/102724676 kt tunstall', 'https://www.bandsintown.com/e/102724676 kt tunstall', 'https://www.bandsintown.com/e/103102900 peter white', 'https://www.bandsintown.com/e/103102900 peter white', 'https://www.bandsintown.com/e/103102900 peter white', 'https://www.bandsintown.com/e/103102900 peter white', 'https://www.bandsintown.com/e/102977409 borgeous', 'https://www.bandsintown.com/e/102977409 borgeous', 'https:

In [90]:
# select the band name after the space
# this should work even for bands that start with numbers!
pattern = re.compile(r"\s(.*)")
reg_list = [re.search(pattern , str(x)).group(0) for x in reg_list]

print(reg_list)

[' martin sexton', ' martin sexton', ' martin sexton', ' martin sexton', ' rebelution', ' rebelution', ' rebelution', ' rebelution', ' kt tunstall', ' kt tunstall', ' kt tunstall', ' kt tunstall', ' peter white', ' peter white', ' peter white', ' peter white', ' borgeous', ' borgeous', ' borgeous', ' borgeous', ' mike love', ' mike love', ' mike love', ' mike love', ' jvna', ' jvna', ' jvna', ' jvna', ' anuhea', ' anuhea', ' anuhea', ' sullivan king', ' sullivan king', ' sullivan king', ' air supply', ' air supply', ' air supply', ' air supply', ' taylor dayne', ' taylor dayne', ' taylor dayne', ' zeds dead', ' zeds dead', ' zeds dead', ' zeds dead', ' fia', ' fia', ' fia', ' fia', ' ron artis ii', ' ron artis ii', ' ron artis ii', ' ron artis ii', ' hippie sabotage', ' hippie sabotage', ' hippie sabotage', ' hippie sabotage', ' krewella', ' krewella', ' krewella', ' krewella', ' henry kapono', ' henry kapono', ' henry kapono', ' henry kapono', ' ajr', ' ajr', ' ajr', ' ajr', ' rebelut

In [91]:
# get rid of parentheses and commas (and nested lists by default)

reg_list = [x.replace("'",'').replace(',', '') for x in reg_list]
print(reg_list)

[' martin sexton', ' martin sexton', ' martin sexton', ' martin sexton', ' rebelution', ' rebelution', ' rebelution', ' rebelution', ' kt tunstall', ' kt tunstall', ' kt tunstall', ' kt tunstall', ' peter white', ' peter white', ' peter white', ' peter white', ' borgeous', ' borgeous', ' borgeous', ' borgeous', ' mike love', ' mike love', ' mike love', ' mike love', ' jvna', ' jvna', ' jvna', ' jvna', ' anuhea', ' anuhea', ' anuhea', ' sullivan king', ' sullivan king', ' sullivan king', ' air supply', ' air supply', ' air supply', ' air supply', ' taylor dayne', ' taylor dayne', ' taylor dayne', ' zeds dead', ' zeds dead', ' zeds dead', ' zeds dead', ' fia', ' fia', ' fia', ' fia', ' ron artis ii', ' ron artis ii', ' ron artis ii', ' ron artis ii', ' hippie sabotage', ' hippie sabotage', ' hippie sabotage', ' hippie sabotage', ' krewella', ' krewella', ' krewella', ' krewella', ' henry kapono', ' henry kapono', ' henry kapono', ' henry kapono', ' ajr', ' ajr', ' ajr', ' ajr', ' rebelut

In [92]:
# drop duplicates by using a set
band_list = list(set(reg_list))
print(band_list)

[' christafari', ' henry kapono', ' martin sexton', ' taylor dayne', ' mike love', ' peter white', ' michael delara', ' jasmin nicole', ' fia', ' ajr', ' bobby moderow jr.', ' kt tunstall', ' avion blackman', ' jvna', ' john craigie', ' rebelution', ' borgeous', ' zeds dead', ' anuhea', ' krewella', ' hippie sabotage', ' air supply', ' ron artis ii', ' sullivan king', ' gabriel iglesias']


In [93]:
load_dotenv()
# grab environment variables
SPOTIFY_CLIENT_ID = os.getenv('SPOTIFY_CLIENT_ID')
SPOTIFY_CLIENT_SECRET = os.getenv('SPOTIFY_CLIENT_SECRET')
REDIRECT_URI = os.getenv('REDIRECT_URI')

In [94]:
sp = spotipy.Spotify(
    auth_manager=SpotifyOAuth(
        scope="playlist-modify-public",
        redirect_uri=REDIRECT_URI,
        client_id=SPOTIFY_CLIENT_ID,
        client_secret=SPOTIFY_CLIENT_SECRET,
        show_dialog=True,
        cache_path="token.txt",

    )
)
user_id = sp.current_user()["id"]

In [95]:
# Creating a new Playlist
playlist = sp.user_playlist_create(
    user=user_id,
    name="Upcoming Local Shows Playlist",
    public=True,
    collaborative=False,
    description="Songs from artists that will be playing in your area soon",
)

playlist_id = playlist["id"]

In [96]:
def grab_track_ids(band_list):

    track_ids = []

    for i in range(len(band_list)):
        results = sp.search(q=f"{band_list[i]}", limit=5, type='track')
        if results['tracks']['total'] == 0:
            continue
        else:
            for j in range(len(results['tracks']['items'])):
                if fuzz.partial_ratio(results['tracks']['items'][j]['artists'][0]['name'], band_list[i]) > 75:
                    track_ids.append(results['tracks']['items'][j]['id'])
                else:
                    continue
    print("Gottem!")

    return track_ids

In [97]:
track_ids = grab_track_ids(band_list)

track_ids

Gottem!


['2ZlaPNCTkfbyOn29z3DrSo',
 '6aBhKc5wZiR8RMgNsc00l0',
 '4pYhtE56PsIbPYLCBTE6T4',
 '1i7RHPlMTpObShchH5OdXL',
 '7w7kCyJkJ0Odr5GdQepVpl',
 '3kNYQNKWBSSDxuUnyers75',
 '404boHKfhWyp7Ez1kg5Uq3',
 '219kkayTVqGSVU7j3cuBuN',
 '3hEQyEnyiyU0l1sgGHMXFp',
 '1SkfMlMAglPbsipBUBMG1H',
 '3IHMlWNnTqtVP32p46ZzJT',
 '3YVPpVJvJG2EauXurPA90F',
 '6jJCk7oHNwyowOUaF3gXEu',
 '3HLgjHYaEhKFvV4SSvI9Te',
 '6enR2PAZd7LUMMwQb4ciuh',
 '4hbXvRJfgctBgz7PVBtWQu',
 '73gASYnzwZAzq5IYQf4Hff',
 '6JXcrPDlgWcgCMZYJQgtTJ',
 '4x8mcYVZNcwmTvpgP74zTi',
 '5tuPqvXQw7SJwuX78R9pB2',
 '6Uob2aJYqcqPv5Z2BigRg6',
 '6yafxZlCDHvMjZYGgoW6zq',
 '1gYfwTmrdMapmkQg58JoW7',
 '0ssLwE3jh5gEeFQ7gfhA7j',
 '0B2H6GWmg8KpBv8qh7uO1S',
 '1DANjn6DqWd98WIR6JfkSp',
 '0eJ55ewXpVQuRqFG8keqf2',
 '07M5jP5WGeCLWhpRVZ7ouL',
 '0a40dbixRhznG359jQSivU',
 '27ytYULTu6QSZBhGaOKq9i',
 '5e6kMwY5gD9sYEEMUl4RbU',
 '2tvhVwzotuuzmRRvbUi6Rb',
 '52Cc6qv2MckQgLjeR5Eai0',
 '0exVC0ue4dRE2FRBFCRG4w',
 '2EDRUNdur7CGV2W2mwIRoS',
 '7fs5E0E7GpjX5Y381D6Rpc',
 '4TyCnstYu6LGrjka5WW6ft',
 

In [99]:
list_of_playlists = sp.user_playlists(user_id, limit=50, offset=0)

for p in list_of_playlists['items']:

    if p['name'] == 'Upcoming Local Shows Playlist':
        p_id = p['id']
        sp.user_playlist_replace_tracks(user_id, p_id, track_ids)
        break
    else:
        # create a new playlist
        playlist = sp.user_playlist_create(user=user_id,
                                name="Upcoming Local Shows Playlist",
                                public=True,
                                collaborative=False,
                                description="Songs from artists that will be playing in your area this coming week")
        # grab playlist id
        p_id = playlist["id"]

        # add tracks to new playlist
        sp.user_playlist_add_tracks(user_id, p_id, track_ids)
